In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import pandas as pd
import re
import time

tfe.enable_eager_execution()

In [3]:
#기본 값
df_path = './dataset'
buckets = [(12, 40)]
PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3
user_vocab_size = 9687
bot_vocab_size = 12169

# 1. Load Dataset

Data from cornell dialogue: https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

In [4]:
# Load the data
lines = open(df_path + '/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open(df_path + '/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [5]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
# Create a list of all of the conversations' lines' ids.
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

In [6]:
# Check if we have loaded the data correctly
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [8]:
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221616
221616


In [9]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [12]:
# Take a look at some of the data to ensure that it has been cleaned well.
limit = 0
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [ ]:
# Find the length of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

## Preprocessing Dataset

- Datset API사용 (큐 기반)
- Clear Structure and Simplicity

In [13]:
def model_inputs():
    
    input_data = tf.placeholder(tf.int32, 
                                [None, None], 
                                name='input')
    
    targets = tf.placeholder(tf.int32, 
                             [None, None], 
                             name='targets')
    
    lr = tf.placeholder(tf.float32, name='learning_rate')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return input_data, targets, lr, keep_prob

def process_encoding_input(target_data, vocab_to_int, batch_size):
    
    ending = tf.strided_slice(target_data, 
                              [0, 0], 
                              [batch_size, -1], 
                              [1, 1]) #마지막에 있는 단어 토큰을 제거함
    
    dec_input = tf.concat([tf.fill([batch_size, 1], 
                                   vocab_to_int['<GO>']), 
                           ending], 1)
    return dec_input

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   sequence_length, attn_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(
               lstm, 
               input_keep_prob = keep_prob)
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(
               cell_fw = enc_cell,
               cell_bw = enc_cell,
               sequence_length = sequence_length,
               inputs = rnn_inputs, 
               dtype=tf.float32)
    return enc_state

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    
    attention_states = tf.zeros([batch_size, 
                                1, 
                                dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
        tf.contrib.seq2seq.prepare_attention(
             attention_states,
             attention_option="bahdanau",
             num_units=dec_cell.output_size)
    
    train_decoder_fn = \  
       tf.contrib.seq2seq.attention_decoder_fn_train(
             encoder_state[0],
             att_keys,
             att_vals,
             att_score_fn,
             att_construct_fn,
             name = "attn_dec_train")
    
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(
             dec_cell, 
             train_decoder_fn, 
             dec_embed_input, 
             sequence_length, 
             scope=decoding_scope)
    
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    
    return output_fn(train_pred_drop)

In [36]:
#Load Sample Dataset

def read_train_set(train_path, max_size=None):
    train_set = [[] for _ in range(buckets[0][0])]
    
    with open(train_path, "r", encoding="utf-8") as ts:
        counter = 0
        while not max_size or counter < max_size:
            
            # iterator 성질을 이용하여, 유저, 봇 발화 불러오기
            user_tokens, bot_tokens = ts.readline(), ts.readline()
            
            # 유저 발화에서 모르는 토큰이 있으면 언노운 아이디 부여한다
            user_token_ids = [int(x) for x in user_tokens.split()]

            # 모르는 토큰이 있으면 언노운 아이디 부여
            for i, user_token_id in enumerate(user_token_ids):
                if user_token_id > user_vocab_size:
                    user_token_ids[i] = UNK_ID

            # 유저 버킷 크기로 잘라낸다
            user_token_bucket = user_token_ids[:buckets[0][0]]
            user_token_len = len(user_token_bucket)

            # 봇 발화에서 모르는 토큰이 있으면 언노운 아이디 부여한다
            bot_token_ids = [int(x) for x in bot_tokens.split()]
            for i, bot_token_id in enumerate(bot_token_ids):
                if bot_token_id > bot_vocab_size:
                    bot_token_ids[i] = UNK_ID

            # 봇 버킷 크기로 잘라낸다
            bot_token_bucket = bot_token_ids[:(buckets[0][1]-1)]

            # 유저 토큰 길이에 따라 유저 발화와 봇 발화를 모은다
            train_set[user_token_len - 1].append([user_token_bucket, bot_token_bucket])

            counter += 1

            # 진행도를 알아볼 수 있도록 1000번마다 프린트한다
            if counter % 1000 == 0:
                print("read data line %d" % counter)

        return train_set
    
train_set = read_train_set(df_path, 5000)
train_len_hist = [len(train_set[i]) for i in range(len(train_set))]
train_size = float(sum(train_len_hist))

print("sep by train, {}".format(train_len_hist))
print("train_size: {}".format(train_size))

read data line 1000
read data line 2000
read data line 3000
read data line 4000
read data line 5000
sep by train, [0, 0, 20, 12, 35, 91, 180, 236, 229, 144, 76, 3977]
train_size: 5000.0


In [50]:
# train_set[4][0][0] #고객 발화
# train_set[4][0][1] #봇 발화

df_train = [train_set[4][i] for i in range(len(train_set[4]))] #Sample로 input size 가 5인 발화를 추출한다.
df_user = [df_train[i][0] for i in range(len(df_train))]
df_bot = [df_train[i][1] for i in range(len(df_train))]

35

In [63]:
embed_size = 1
window_size = 3
dict_size = 5
filter_size = window_size*embed_size

ids = tf.constant(df_train)

#https://tutorials.botsfloor.com/how-to-build-your-first-chatbot-c84495d4622d



def input_fn():
    ids = tf.constant([[5, 6, 10, 9, 2], [5, 6, 10, 60, 2]])
    label_ids = tf.constant([[4, 22, 163, 649, 2], [4, 22, 163, 649, 2]])
    return {'ids': ids,
            'init_label': label_ids[:, :window_size],
            'label_length': int(label_ids.shape[1])-window_size
            }, label_ids[:, window_size:]

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [3]:
ids = tf.constant([[1, 2, 3, 4, 3, 2, 1], [1, 2, 3, 2, 3, 2, 3]])
label_ids = tf.constant([[0, 0, 0, 1, 3, 1, 3, 2], [0, 0, 0, 2, 1, 3, 4, 4]]) #무조건 앞에 3개를... 
init_label = label_ids[:, :window_size]
label_length = int(label_ids.shape[1]) - window_size
label_ids = label_ids[:, window_size:]

print(init_label)
print(label_length)
print(label_ids)

tf.Tensor(
[[0 0 0]
 [0 0 0]], shape=(2, 3), dtype=int32)
5
tf.Tensor(
[[1 3 1 3 2]
 [2 1 3 4 4]], shape=(2, 5), dtype=int32)


In [40]:
embeddings = tfe.Variable(tf.truncated_normal([dict_size, embed_size]), trainable=False)
print(embeddings.shape)

(5, 10)


In [4]:
class Embedding(tf.layers.Layer):
    """ 임베딩 층"""
    def __init__(self, vocab_size, embedding_dim, **kwargs):
        super(Embedding, self).__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
    def build(self, _):
        self.embedding = self.add_variable(
        "embedding_kernel",
        shape=[self.vocab_size, self.embedding_dim],
        dtype=tf.float32,
        initializer = tf.random_uniform_initializer(-0.1, 0.1),
        trainable=True)
    
    def call(self, x):
        return tf.nn.embedding_lookup(self.embedding, x)
    

Embedding(50, 100)

In [50]:
# embedding
input_embeds = tf.nn.embedding_lookup(embeddings, ids)
print(input_embeds)

input_flat = tf.layers.flatten(input_embeds)
input_flat = tf.expand_dims(input_flat, -1)

init_label = tf.nn.embedding_lookup(embeddings, init_label)
init_label = tf.layers.flatten(init_label)
init_label = tf.expand_dims(init_label, -1)

print(input_flat.shape, init_label.shape)

label_onehot = tf.one_hot(label_ids, depth=dict_size, dtype=tf.float32)
print(label_onehot)

tf.Tensor(
[[[-0.44031993  0.27558795  1.88564384 -0.26049265  1.04315972 -0.8651666
    0.45365837  0.21361713  0.36355501  1.71674705]
  [ 0.00447189 -0.20288323 -0.78719151  0.80474299 -0.97947961  0.43934852
   -0.00413364  0.87714851 -0.58071738  0.23394325]
  [-1.54866028  0.79748154  0.61276042  0.14294747  1.46765685  1.49111998
   -1.82410061  0.7496416   0.30667016  1.82444489]
  [-1.24891686  1.56246781 -1.05082929 -0.79415643 -0.33950487 -1.42375171
   -0.82772219  0.00675092 -0.51040632  1.12121177]
  [-1.54866028  0.79748154  0.61276042  0.14294747  1.46765685  1.49111998
   -1.82410061  0.7496416   0.30667016  1.82444489]
  [ 0.00447189 -0.20288323 -0.78719151  0.80474299 -0.97947961  0.43934852
   -0.00413364  0.87714851 -0.58071738  0.23394325]
  [-0.44031993  0.27558795  1.88564384 -0.26049265  1.04315972 -0.8651666
    0.45365837  0.21361713  0.36355501  1.71674705]]

 [[-0.44031993  0.27558795  1.88564384 -0.26049265  1.04315972 -0.8651666
    0.45365837  0.21361713

InvalidArgumentError: Value for attr 'Tindices' of float is not in the list of allowed values: int32, int64
	; NodeDef: ResourceGather = ResourceGather[Tindices=DT_FLOAT, dtype=DT_FLOAT, validate_indices=true](dummy_input, dummy_input); Op<name=ResourceGather; signature=resource:resource, indices:Tindices -> output:dtype; attr=validate_indices:bool,default=true; attr=dtype:type; attr=Tindices:type,allowed=[DT_INT32, DT_INT64]; is_stateful=true> [Op:ResourceGather] name: embedding_lookup/embedding_lookup//

In [55]:
# encoder
encoder_conv = tf.layers.conv1d(
        inputs=input_flat,
        filters=2*embed_size,
        kernel_size=filter_size,
        strides=embed_size,
        padding='same')

print(encoder_conv.shape)
#embed_size, 반으로 나눈다.
encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])
print(encoder_glu.shape)

(2, 7, 20)
(2, 7, 10)


In [64]:
#decoder: 그림에서 왼쪽에서 하나의 블록을 구하는 식

decoder_conv = tf.layers.conv1d(
        inputs=init_label,
        filters=2*embed_size,
        kernel_size=filter_size,
        strides=embed_size)

decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

print(decoder_conv.shape)
print(decoder_glu.shape)

tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1]) #decoder 단어 1개, encoder는 7개단어)
print(tiled_decoder_glu.shape)

dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)
print(dot_prod.shape)

(2, 1, 20)
(2, 1, 10)
(2, 7, 10)
(2, 7, 1)
tf.Tensor(
[[[ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]]

 [[ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]]], shape=(2, 7, 1), dtype=float32)


In [90]:
attention = tf.nn.softmax(dot_prod, axis=1) #수정해야함
#tf.reduce_sum(attention, axis=1)
print(attention.shape)

z_plus_e = encoder_glu + input_embeds #why???
print(z_plus_e.shape)

tiled_attention = tf.tile(attention, [1, 1, embed_size])
print(tiled_attention.shape)

c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
print(c.shape)

decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])
print(decoder_glu.shape)

logits = tf.layers.dense(c+decoder_glu, dict_size)
print(logits.shape)

out = tf.nn.softmax(logits)
print(out)

next_id = tf.argmax(out, axis=1)
next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
next_embeds = tf.expand_dims(next_embeds, -1)
decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

next_ids.append(next_id)
outs.append(logits)

(2, 7, 1)
(2, 7, 10)
(2, 7, 10)
(2, 10)
(2, 10)
(2, 5)
tf.Tensor(
[[ 0.27388293  0.11168921  0.34511361  0.04551498  0.22379932]
 [ 0.37078524  0.07891139  0.3009578   0.02622434  0.2231212 ]], shape=(2, 5), dtype=float32)


NameError: name 'decoder_input' is not defined

In [65]:
c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

logits = tf.layers.dense(c+decoder_glu, dict_size)

out = tf.nn.softmax(logits) #out으로 묶기

next_id = tf.argmax(out, axis=1)
next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
next_embeds = tf.expand_dims(next_embeds, -1)
decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

next_ids.append(next_id)
outs.append(logits)

outs = tf.stack(outs, axis=1)
next_ids = tf.stack(next_ids, axis=1)

if TRAIN:
    global_step = tf.train.get_global_step()
    loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
    train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
    estimator_spec = tf.estimator.EstimatorSpec(
            mode=mode,
            train_op=train_op,
            loss=loss)

tf.Tensor(
[[[ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]]

 [[ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]
  [ 1.]]], shape=(2, 7, 1), dtype=float32)


In [93]:
#decoder
next_ids = []
outs = []
for l in range(label_length):
    if l == 0: decoder_input = init_label

    decoder_conv = tf.layers.conv1d(
            inputs=init_label,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size)


    decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

    tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])


    dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

    attention = tf.nn.softmax(dot_prod)

    z_plus_e = encoder_glu + input_embeds

    tiled_attention = tf.tile(attention, [1, 1, embed_size])

    c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
    decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

    logits = tf.layers.dense(c+decoder_glu, dict_size)

    out = tf.nn.softmax(logits)

    next_id = tf.argmax(out, axis=1)
    next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
    next_embeds = tf.expand_dims(next_embeds, -1)
    decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1) #옆에 붙이자, 기존꺼 버리고..

    next_ids.append(next_id)
    outs.append(logits)

outs = tf.stack(outs, axis=1)
next_ids = tf.stack(next_ids, axis=1)

print(outs)
print(next_ids)

tf.Tensor(
[[[ 1.60669613  0.58048368  4.32952166 -0.75439    -6.57604551]
  [ 2.47091866 -1.72941756 -4.18986034  2.36932921  2.30357647]
  [ 2.63007236 -6.39063406 -1.39494836 -2.52273297  6.33367109]
  [ 6.15402555 -5.65982628 -2.03500366  2.70822239  1.98549938]
  [ 0.19027537  5.78388309  2.20064783 -3.58040476  1.57070911]]

 [[ 0.87436223 -1.44449735  6.2491827   2.25679016 -9.77695084]
  [-0.18215799 -1.39346814  0.43136549  5.30196714  1.92673659]
  [ 4.68214655 -7.55460072 -5.41391659 -4.77869511  9.60904503]
  [ 7.43560982 -3.8282907  -1.15743423  0.38468164 -0.97345304]
  [ 1.20879579  9.34952164  1.27624547 -1.68866849 -2.31034088]]], shape=(2, 5, 5), dtype=float32)
tf.Tensor(
[[2 0 4 0 1]
 [2 3 4 0 1]], shape=(2, 5), dtype=int64)


In [95]:
loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

RuntimeError: `loss` passed to Optimizer.compute_gradients should be a function when eager execution is enabled.

In [ ]:
encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])

#decoder
next_ids = []
outs = []
for l in range(features['label_length']):
    if l == 0: decoder_input = init_label

    decoder_conv = tf.layers.conv1d(
            inputs=init_label,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size)


    decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

    tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])


    dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

    attention = tf.nn.softmax(dot_prod)

    z_plus_e = encoder_glu + input_embeds

    tiled_attention = tf.tile(attention, [1, 1, embed_size])

    c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
    decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

    logits = tf.layers.dense(c+decoder_glu, dict_size)

    out = tf.nn.softmax(logits)

    next_id = tf.argmax(out, axis=1)
    next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
    next_embeds = tf.expand_dims(next_embeds, -1)
    decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1) #옆에 붙이자, 기존꺼 버리고..

    next_ids.append(next_id)
    outs.append(logits)

outs = tf.stack(outs, axis=1)
next_ids = tf.stack(next_ids, axis=1)

if TRAIN:
    global_step = tf.train.get_global_step()
    loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
    train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
    estimator_spec = tf.estimator.EstimatorSpec(
            mode=mode,
            train_op=train_op,
            loss=loss)


In [36]:
def model_fn(mode, features, labels):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embeddings = tf.Variable(tf.truncated_normal([dict_size, embed_size]), trainable=False)

    # embedding
    input_embeds = tf.nn.embedding_lookup(embeddings, features['ids'])

    input_flat = tf.layers.flatten(input_embeds)
    input_flat = tf.expand_dims(input_flat, -1)

    init_label = tf.nn.embedding_lookup(embeddings, features['init_label'])
    init_label = tf.layers.flatten(init_label)
    init_label = tf.expand_dims(init_label, -1)

    if not PREDICT:
        label_onehot = tf.one_hot(labels, depth=dict_size, dtype=tf.float32)

    # encoder
    encoder_conv = tf.layers.conv1d(
            inputs=input_flat,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size,
            padding='same')

    encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])



        decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

        tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])


        dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

        attention = tf.nn.softmax(dot_prod)

        z_plus_e = encoder_glu + input_embeds

        tiled_attention = tf.tile(attention, [1, 1, embed_size])

        c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
        decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

        logits = tf.layers.dense(c+decoder_glu, dict_size)

        out = tf.nn.softmax(logits)

        next_id = tf.argmax(out, axis=1)
        next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
        next_embeds = tf.expand_dims(next_embeds, -1)
        decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

        next_ids.append(next_id)
        outs.append(logits)

    outs = tf.stack(outs, axis=1)
    next_ids = tf.stack(next_ids, axis=1)

    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
        train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                train_op=train_op,
                loss=loss)

    elif EVAL:
        loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
        eval_metric_ops = {'acc': tf.metrics.accuracy(labels, next_ids)}
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=loss,
                eval_metric_ops=eval_metric_ops)

    elif PREDICT:
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={'prediction': next_ids})

    else:
        raise Exception('estiamtor spec is invalid')

    return estimator_spec


def main(_):
    fairseq = tf.estimator.Estimator(model_fn)
    fairseq.train(input_fn, steps=10000)
    fairseq.evaluate(input_fn, steps=10)

    pred = fairseq.predict(input_fn)
    for p in pred:
        print('prediction: {}'.format(p['prediction']))

if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run()

FATAL Flags parsing error: Unknown command line flag 'f'
Pass --helpshort or --helpfull to see help on flags.


SystemExit: 1

/Users/naver/tf_nightly/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
